In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Tuple

# KV 캐시의 동작을 보여주기 위한 최소한의 어텐션 모듈
class SimpleAttention(nn.Module):
    def __init__(self, d_model: int, n_heads: int):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        
        self.qkv_proj = nn.Linear(d_model, d_model * 3)
        self.o_proj = nn.Linear(d_model, d_model)

    def forward(self, x: torch.Tensor, past_kv: Optional[Tuple[torch.Tensor, torch.Tensor]] = None):
        batch_size, seq_len, _ = x.shape

        # 1. Q, K, V 계산
        q, k, v = self.qkv_proj(x).chunk(3, dim=-1)
        
        # Multi-head 형태로 변환
        q = q.view(batch_size, seq_len, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, seq_len, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, seq_len, self.n_heads, self.head_dim).transpose(1, 2)

        # 2. KV 캐시 처리
        if past_kv is not None:
            # Decoding 단계: 과거의 K, V를 가져와 현재 K, V와 연결
            past_k, past_v = past_kv
            k = torch.cat([past_k, k], dim=2)
            v = torch.cat([past_v, v], dim=2)
        
        # 현재 스텝까지의 K, V를 다음 스텝으로 전달하기 위해 저장
        present_kv = (k, v)

        # 3. 어텐션 계산
        attn_output = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        
        output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        
        return self.o_proj(output), present_kv

# 간단한 모델
class SimpleTransformer(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.embedding = nn.Embedding(100, d_model) # vocab_size=100
        self.attention = SimpleAttention(d_model, n_heads)
        self.lm_head = nn.Linear(d_model, 100) # vocab_size=100
        
    def forward(self, tokens, past_kv=None):
        x = self.embedding(tokens)
        x, present_kv = self.attention(x, past_kv)
        logits = self.lm_head(x)
        return logits, present_kv

if __name__ == '__main__':
    d_model = 64
    n_heads = 4
    batch_size = 1
    
    model = SimpleTransformer(d_model, n_heads)
    model.eval()

    print("--- KV 캐시 동작 과정 시뮬레이션 ---")

    # --- 1단계: Prefill (프롬프트 처리) ---
    print("\n[1. Prefill 단계]")
    prompt_tokens = torch.randint(0, 100, (batch_size, 10)) # (B, 10) 크기의 프롬프트
    print(f"입력 프롬프트 shape: {prompt_tokens.shape}")

    with torch.no_grad():
        logits, kv_cache = model(prompt_tokens)
    
    # kv_cache[0]은 Key, kv_cache[1]은 Value
    print(f"생성된 KV 캐시 shape: K={kv_cache[0].shape}, V={kv_cache[1].shape}")
    print("-> 프롬프트 10개 토큰에 대한 K, V가 캐시에 저장되었습니다.")

    # --- 2단계: Decoding (다음 토큰 생성) ---
    print("\n[2. Decoding 단계 (1-step)]")
    # 이제부터는 단 하나의 토큰만 입력으로 넣습니다.
    next_token = torch.randint(0, 100, (batch_size, 1))
    print(f"입력 토큰 shape: {next_token.shape}")
    print(f"전달되는 과거 KV 캐시 shape: K={kv_cache[0].shape}, V={kv_cache[1].shape}")
    
    with torch.no_grad():
        # 이전 스텝의 캐시(kv_cache)를 past_kv로 전달합니다.
        new_logits, new_kv_cache = model(next_token, past_kv=kv_cache)

    print(f"업데이트된 KV 캐시 shape: K={new_kv_cache[0].shape}, V={new_kv_cache[1].shape}")
    print("-> 새로운 토큰 1개에 대한 K, V가 기존 캐시에 추가되어 길이가 11이 되었습니다.")

    print("\n[2. Decoding 단계 (2-step)]")
    next_token_2 = torch.randint(0, 100, (batch_size, 1))
    print(f"입력 토큰 shape: {next_token_2.shape}")
    print(f"전달되는 과거 KV 캐시 shape: K={new_kv_cache[0].shape}, V={new_kv_cache[1].shape}")
    
    with torch.no_grad():
        _, final_kv_cache = model(next_token_2, past_kv=new_kv_cache)

    print(f"업데이트된 KV 캐시 shape: K={final_kv_cache[0].shape}, V={final_kv_cache[1].shape}")
    print("-> 다시 새로운 토큰 1개에 대한 K, V가 추가되어 길이가 12가 되었습니다.")


--- KV 캐시 동작 과정 시뮬레이션 ---

[1. Prefill 단계]
입력 프롬프트 shape: torch.Size([1, 10])
생성된 KV 캐시 shape: K=torch.Size([1, 4, 10, 16]), V=torch.Size([1, 4, 10, 16])
-> 프롬프트 10개 토큰에 대한 K, V가 캐시에 저장되었습니다.

[2. Decoding 단계 (1-step)]
입력 토큰 shape: torch.Size([1, 1])
전달되는 과거 KV 캐시 shape: K=torch.Size([1, 4, 10, 16]), V=torch.Size([1, 4, 10, 16])
업데이트된 KV 캐시 shape: K=torch.Size([1, 4, 11, 16]), V=torch.Size([1, 4, 11, 16])
-> 새로운 토큰 1개에 대한 K, V가 기존 캐시에 추가되어 길이가 11이 되었습니다.

[2. Decoding 단계 (2-step)]
입력 토큰 shape: torch.Size([1, 1])
전달되는 과거 KV 캐시 shape: K=torch.Size([1, 4, 11, 16]), V=torch.Size([1, 4, 11, 16])
업데이트된 KV 캐시 shape: K=torch.Size([1, 4, 12, 16]), V=torch.Size([1, 4, 12, 16])
-> 다시 새로운 토큰 1개에 대한 K, V가 추가되어 길이가 12가 되었습니다.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Tuple
import time

# GPU 사용 가능 여부 확인 및 디바이스 설정
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"사용 디바이스: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("!!! 경고: 이 성능 시연은 GPU 환경에서 실행해야 유의미한 차이를 보입니다. !!!")

# 이전 예제와 동일한 간단한 어텐션 및 모델 정의
class SimpleAttention(nn.Module):
    def __init__(self, d_model: int, n_heads: int):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        self.qkv_proj = nn.Linear(d_model, d_model * 3)
        self.o_proj = nn.Linear(d_model, d_model)

    def forward(self, x: torch.Tensor, past_kv: Optional[Tuple[torch.Tensor, torch.Tensor]] = None):
        batch_size, seq_len, _ = x.shape
        q, k, v = self.qkv_proj(x).chunk(3, dim=-1)
        q = q.view(batch_size, seq_len, self.n_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, seq_len, self.n_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, seq_len, self.n_heads, self.head_dim).transpose(1, 2)

        if past_kv is not None:
            past_k, past_v = past_kv
            k = torch.cat([past_k, k], dim=2)
            v = torch.cat([past_v, v], dim=2)
        
        present_kv = (k, v)
        attn_output = F.scaled_dot_product_attention(q, k, v, is_causal=(past_kv is None))
        output = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        return self.o_proj(output), present_kv

class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, d_model, n_heads):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.attention = SimpleAttention(d_model, n_heads)
        self.lm_head = nn.Linear(d_model, vocab_size)
        
    def forward(self, tokens, past_kv=None):
        x = self.embedding(tokens)
        x, present_kv = self.attention(x, past_kv)
        logits = self.lm_head(x)
        return logits, present_kv

# --- 생성 함수 정의 ---

def generate_with_cache(model, prompt, n_tokens_to_gen):
    """KV 캐시를 사용하여 토큰을 생성하는 효율적인 방법"""
    model.eval()
    kv_cache = None
    generated_tokens = prompt
    
    # 1. Prefill
    with torch.no_grad():
        _, kv_cache = model(prompt, past_kv=None)
    
    # 2. Decoding
    next_token = torch.argmax(model(prompt, past_kv=None)[0][:, -1, :], dim=-1, keepdim=True)
    
    for _ in range(n_tokens_to_gen - 1):
        with torch.no_grad():
            logits, kv_cache = model(next_token, past_kv=kv_cache)
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
            generated_tokens = torch.cat([generated_tokens, next_token], dim=1)
            
    return generated_tokens

def generate_without_cache(model, prompt, n_tokens_to_gen):
    """KV 캐시를 사용하지 않는 비효율적인 방법"""
    model.eval()
    generated_tokens = prompt
    
    for _ in range(n_tokens_to_gen):
        with torch.no_grad():
            # 매번 전체 시퀀스를 다시 입력
            logits, _ = model(generated_tokens, past_kv=None)
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
            generated_tokens = torch.cat([generated_tokens, next_token], dim=1)
            
    return generated_tokens

# --- 성능 측정 ---

if __name__ == '__main__':
    # 모델 및 설정
    vocab_size = 50257
    d_model = 512
    n_heads = 8
    
    model = SimpleTransformer(vocab_size, d_model, n_heads).to(device)

    prompt_len = 128
    n_gen = 128
    prompt = torch.randint(0, vocab_size, (1, prompt_len)).to(device)

    print("\n--- KV 캐시 유무에 따른 추론 속도 비교 ---")
    print(f"프롬프트 길이: {prompt_len}, 생성할 토큰 수: {n_gen}\n")

    # 워밍업
    for _ in range(3):
        generate_with_cache(model, prompt, 10)
        generate_without_cache(model, prompt, 10)
    if torch.cuda.is_available():
        torch.cuda.synchronize()

    # KV 캐시 사용 성능 측정
    t0 = time.time()
    generate_with_cache(model, prompt, n_gen)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    t1 = time.time()
    time_with_cache = t1 - t0
    print(f"✅ KV 캐시 사용 시: {time_with_cache:.4f} 초")

    # KV 캐시 미사용 성능 측정
    t0 = time.time()
    generate_without_cache(model, prompt, n_gen)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    t1 = time.time()
    time_without_cache = t1 - t0
    print(f"❌ KV 캐시 미사용 시: {time_without_cache:.4f} 초")

    # 결과 비교
    speedup = time_without_cache / time_with_cache
    print(f"\n🚀 성능 향상: KV 캐시 사용 시 약 {speedup:.2f}배 더 빠릅니다.")


사용 디바이스: NVIDIA A100 80GB PCIe

--- KV 캐시 유무에 따른 추론 속도 비교 ---
프롬프트 길이: 128, 생성할 토큰 수: 128

✅ KV 캐시 사용 시: 0.0315 초
❌ KV 캐시 미사용 시: 0.0997 초

🚀 성능 향상: KV 캐시 사용 시 약 3.17배 더 빠릅니다.
